##Web crawling code

In [ ]:
#pip install selenium

#Panknshop


In [ ]:
# parknshop web crawler

import pandas as pd
from selenium import webdriver # use webdriver
from selenium.webdriver.common.by import By # different methods of locating data
from selenium.webdriver.chrome.options import Options # options for selenium driver
import time
import datetime



def crawler(url):

    driver = webdriver.Chrome() # create instance of selenium driver

    try:
        driver.get(url)
    except:
        print("url not found")
        exit

    time.sleep(2)


    # get the total number of products (shown on the web page)              XPath of no of product
    total_no_of_product = driver.find_element(By.XPATH, "/html/body/app-root/cx-storefront/main/cx-page-layout/cx-page-slot//pns-product-list/div[1]/div[1]")
    total_no_of_product = int(total_no_of_product.text.split(" ")[0])       # get the total number of products (shown on the web page)
    print('total_no_of_product : ', total_no_of_product)

    # find product elements on website
    products = driver.find_elements(By.XPATH, "//html/body/app-root/cx-storefront/main/cx-page-layout/cx-page-slot//pns-product-list/div[2]/div/pns-product-tile")
    print(len(products))        # print number of product elements found

    # create empty lists for dataframe
    remark = []
    name = []
    capacity = []
    selling_price = []
    original_price = []
    more_info = []
    link = []

    # create dictionary for dataframe
    result_dict = {
        "remark" : remark,
        "name" : name,
        "capacity" : capacity,
        "selling_price" : selling_price,
        "original_price" : original_price,
        "more_info" : more_info,
        "link" : link
    }


    index  = 0      # for checking process
    while len(products) < total_no_of_product:

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")     # scroll to the bottom of the page, for showing more products
        time.sleep(2)
        #  find product elements on website
        products = driver.find_elements(By.XPATH, "//html/body/app-root/cx-storefront/main/cx-page-layout/cx-page-slot//pns-product-list/div[2]/div/pns-product-tile")

        print(f'{index} - {len(products)}')     # for checking process
        index += 1      # for checking process


    for product in products:        # for each product
        # print(product.text.split("\n"))
        # print(len(product.text.split("\n")))
        # print('=' * 80)

        product_info = product.text.split("\n")     # split the product information by "/n",  return a list
        # the following if else condition is trying to input the product information into the correct column of the future dataframe
        if len(product_info) == 4:
            remark.append(None)
            name.append(product_info[0])
            capacity.append(product_info[1])
            selling_price.append(product_info[2])
            original_price.append(None)
            more_info.append(product_info[-1])

        elif len(product_info) == 5:
            if "$" in product_info[-2] and "$" in product_info[-3]:
                remark.append(None)
                name.append(product_info[0])
                capacity.append(product_info[1])
                selling_price.append(product_info[2])
                original_price.append(product_info[3])
            else:
                remark.append(product_info[0])
                name.append(product_info[1])
                capacity.append(product_info[2])
                selling_price.append(product_info[3])
                original_price.append(None)

            more_info.append(product_info[-1])

        elif len(product_info) == 6:
            remark.append(product_info[0])
            name.append(product_info[1])
            capacity.append(product_info[2])
            selling_price.append(product_info[3])
            original_price.append(product_info[4])
            more_info.append(product_info[-1])

    # find all the product link on the page
    product_link = driver.find_elements(By.XPATH, "/html/body/app-root/cx-storefront/main/cx-page-layout/cx-page-slot//pns-product-list/div[2]/div/pns-product-tile//div/div[1]/div[2]/a")
    for product in product_link:
        link.append(product.get_attribute('href'))      # get product link and put it into the list


    driver.quit()       # close the browser

    df = pd.DataFrame(result_dict)      # create dataframe

    # use apply to remove "$" and "," from price by using data_cleaning function
    # df['selling_price'] = df['selling_price'].apply(data_cleaning)
    # df['original_price'] = df['original_price'].apply(data_cleaning)

    return df       # return the dataframe


# replace "HK$" & "," of str price, return float
"""
def data_cleaning(n):
    if type(n) == str:
        n = n.replace("$", "")
        n = n.replace(",", "")
        n = float(n)
    return n
 """


if __name__ == '__main__':
    url = "https://www.pns.hk/zh-hk/%E9%A3%9F%E5%93%81%E5%8F%8A%E9%A3%B2%E5%93%81/%E6%B1%BD%E6%B0%B4/c/04010200"     # 汽水
    current_time_str = datetime.datetime.now().strftime("%Y_%m_%d")    # today's date > datetime obj > str
    df = crawler(url)       # crawler() function will return a dataframe
    print(df)
    df.to_csv(current_time_str + "_demo.csv", encoding="UTF-8")     # save dataframe as csv file


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Welcome

In [ ]:
#Wellcome web crawler

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import datetime

driver = webdriver.Chrome()
current_page= 1
url = "https://www.wellcome.com.hk/zh-hant/category/100002/"+ str(current_page) +".html"

try:
    driver.get(url)
except:
    print('url not found')
    exit

time.sleep(2)

remark=[]
name = []
selling_price=[]
original_price=[]
link=[]
selling_price_base=[]

df_dict={
    "name" : name,
    "selling_price" : selling_price,
    "original_price" : original_price,
    "remark" : remark,
    "link" : link
}


total_page = driver.find_element(By.XPATH,'//a[@class="last cursor num-box"]')
total_page = int(total_page.text)
#print('total page : ', total_page)


while total_page >= current_page:
    url = "https://www.wellcome.com.hk/zh-hant/category/100002/"+ str(current_page) +".html"
    try:
        driver.get(url)

        products = driver.find_elements(By.XPATH,'//a[@class="a-link router-link ware-wrapper"] ')
        for product in products:
        # name= product.find_elements(By.XPATH, './/div[contains(@class, "name")]')[0].text
        # print(name)

        # print(product.text.split('\n'))

            try:
                remark.append(product.find_elements(By.XPATH, './/div[contains(@class, "pro tag")]')[0].text)
            except:
                remark.append(None)

            try:
                name.append(product.find_elements(By.XPATH, './/div[contains(@class, "name")]')[0].text)
            except:
                name.append(None)

            try:
                price_base = product.find_elements(By.XPATH, './/div[contains(@class, "price")]')[0].text.split("\n")[0]
                small_base = product.find_elements(By.XPATH, './/div[contains(@class, "price")]')[0].text.split("\n")[1]
                total=price_base+small_base
                selling_price.append(total)

            except Exception as error:
                selling_price.append(None)
            try:
                original_price.append(product.find_elements(By.XPATH, './/span[contains(@class, "line-price")]')[0].text)
            except:
                original_price.append(None)

        product_link =driver.find_elements(By.XPATH, './/a[contains(@class, "a-link router-link ware-wrapper")]')
        for product in product_link:
            link.append(product.get_attribute('href'))

        current_page = current_page+1
    except:
        print('url not found')
        exit

df = pd.DataFrame(df_dict)

if __name__ == '__main__':
    current_time_str = datetime.datetime.now().strftime("%Y_%m_%d")
    print(df)
    df.to_csv(current_time_str + "_wellcome.csv", encoding="UTF-8")



#Ztore


In [ ]:
#Ztore

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import datetime

def ztore_crawler(url):

    driver = webdriver.Chrome() # create instance of selenium driver

    try:
        driver.get(url)
    except:
        print('url not found')
        exit


    result01 = driver.find_element(By.XPATH, '//*[@id="BaseLayout"]/div/div[4]/div[2]/div')
    print(result01.text)

    #Number of product
    reult02 = driver.find_element(By.XPATH, '//*[@id="BaseLayout"]/div/div[4]/div[1]/div[1]/div[2]/div/div/div[1]/span/span')
    reult02 = reult02.text
    print('reult02 : ', reult02)

    #Click button for show all
    send_button = driver.find_element(By.XPATH, '//*[@id="BaseLayout"]/div/div[4]/span/div/span')
    send_button.click()


    time.sleep(3)


    remark=[]
    name = []
    selling_price=[]
    original_price=[]
    link=[]
    selling_price_base=[]


    result_dict={
        "name" : name,
        "selling_price" : selling_price,
        "original_price" : original_price,
        "remark" : remark,
        "link" : link
    }

    for product in result01:
        try:
            remark.append(product.find_elements(By.XPATH, './/div[contains(@class, "jsx-845759319 label-text")]')[0].text)
        except:
            remark.append(None)
        try:
            name.append(product.find_elements(By.XPATH, './/div[contains(@class, "jsx-2940394913 jsx-676457135")]')[0].text)
        except:
            name.append(None)
        try:
            price_base = product.find_elements(By.XPATH, './/div[contains(@class, "jsx-2940394913 jsx-676457135 promotion")]')[0].text.split("\n")[0]
            selling_price.append(None)

        except Exception as error:
            selling_price.append(None)
        try:
            original_price.append(product.find_elements(By.XPATH, './/span[contains(@class, "jsx-1595401699 PerCountLabel")]')[0].text)
        except:
            original_price.append(None)

        product_link =driver.find_elements(By.XPATH, './/a[contains(@class, "jsx-2940394913 jsx-676457135 ProductItem  windowing-layout")]')
        for product in product_link:
            link.append(product.get_attribute('href'))

    driver.quit()

    return result_dict


ztore_url = {"case_offer":"https://www.ztore.com/tc/category/all/beverage/case-offers",
             "recommended_drinks":"https://www.ztore.com/tc/category/all/beverage/recommended-drinks",
             "lemon_tea":"https://www.ztore.com/tc/category/all/beverage/rtd--tea-lemon-tea",
             "coffee_milk_tea":"https://www.ztore.com/tc/category/all/beverage/rtd-coffee-milk-tea",
             "beverage":"https://www.ztore.com/tc/category/all/beverage/carbonated-beverage",
             "water":"https://www.ztore.com/tc/category/all/beverage/water",
             "juice_energy_drink":"https://www.ztore.com/tc/category/all/beverage/juice-and-energy-drink",
             "healthy_drinks":"https://www.ztore.com/tc/category/all/beverage/healthy-drinks",
             "plant_based_milk":"https://www.ztore.com/tc/category/all/beverage/soy-plant-based-milk",
             "long_life_milk":"https://www.ztore.com/tc/category/all/beverage/long-life-milk-soy",
             "coffee_tea":"https://www.ztore.com/tc/category/all/beverage/coffee-tea",
             "energy_drink":"https://www.ztore.com/tc/category/all/beverage/energy-drink"
             }

if __name__ == '__main__':          #print csvfile
    for k, v in ztore_url.items():
        current_time_str = datetime.datetime.now().strftime("%Y_%m_%d")

        df = pd.DataFrame(ztore_crawler(v))
        df.to_csv(k +  "_" + current_time_str + "ztore.csv", encoding="UTF-8")


